In [8]:
import os
import xarray as xr
import pandas as pd
import numpy as np

# WARNING: see "m00.nc" in the code for Judith

varname = "tas_2m"  # User specifies: ["tas_2m", "pr_sfc"]
method = "allLeads"  # User specifies: ["allLeads", "weeks12lead", "weeks34lead", "weeks45lead"]
calculate_ensemble_mean = True  # User specifies: [True, False]
start_date = "1999-01-01" # User specifies: [eg, 1999-01-01] Note: doesn't need to be a monday
end_date = "2020-12-31"  # User specifies: [eg, 2020-12-31] Note: doesn't need to be a monday
destDir="/glade/campaign/cesm/development/cross-wg/S2S/sglanvil/forJudith/" # User specifies 

# Check the time range if calculate_ensemble_mean is False
if not calculate_ensemble_mean:
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    time_range = (end - start).days
    if time_range > 370:
        print(f"Warning: Your time range is {time_range} days, which exceeds the 370-day limit.")
        print("When choosing to keep all members, you need to shorten your date range to less than 370 days.")
        raise ValueError("Time range exceeds the allowed limit.")  # This will stop execution

base_dir = f"/glade/campaign/cesm/development/cross-wg/S2S/CESM2/S2SHINDCASTS/p1/{varname}"
mondays = pd.date_range(start_date, end_date, freq="W-MON")

methods = {
    "allLeads": (None, np.arange(1, 47)),  # No time slicing, set time coords
    "weeks12lead": (slice(0, 14), None),
    "weeks34lead": (slice(14, 28), None),
    "weeks56lead": (slice(28, 42), None),
}
time_slice, time_coords = methods.get(method, (None, None))

# Initialize result containers
data_results, init_array = [], []

for monday in mondays:
    short_date = monday.strftime("%d%b%Y").lower()
    init_date = int(monday.strftime("%Y%m%d"))
    dir_path = os.path.join(base_dir, str(monday.year), f"{monday.month:02d}")
    print(short_date)
    if not os.path.exists(dir_path): 
        continue
    files = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if f.endswith("m10.nc") and short_date in f]
    if not files:
        continue
    datasets = [xr.open_dataset(fp)[[varname]] for fp in files]
    combined = xr.concat(datasets, dim="member")

    # Apply time slicing if specified
    if time_slice:
        combined = combined.isel(time=time_slice).mean(dim="time")

    # Calculate or keep all ensemble members
    data = combined.mean(dim="member").expand_dims(init=[init_date]) if calculate_ensemble_mean else combined.expand_dims(init=[init_date])

    # Assign time coordinates if available
    if time_coords is not None:
        data = data.assign_coords({"time": time_coords})

    # Store results
    data_results.append(data[varname])
    init_array.append(init_date)

# Combine results across all dates
data_combined = xr.concat(data_results, dim="init")

# Create final dataset
lon_array, lat_array = combined.lon, combined.lat
dataset = xr.Dataset({
    "lon": lon_array,
    "lat": lat_array,
    "init": init_array,
    "data": data_combined,
})

# Save dataset
output_path = f"{destDir}/{varname}_cesm2cam6v2_allLeads_m10.nc"
dataset.to_netcdf(output_path)
output_path = f"{destDir}/{varname}_cesm2cam6v2_allLeads_m10.zarr"
dataset.to_zarr(output_path, mode='w')


04jan1999
11jan1999
18jan1999
25jan1999
01feb1999
08feb1999
15feb1999
22feb1999
01mar1999
08mar1999
15mar1999
22mar1999
29mar1999
05apr1999
12apr1999
19apr1999
26apr1999
03may1999
10may1999
17may1999
24may1999
31may1999
07jun1999
14jun1999
21jun1999
28jun1999
05jul1999
12jul1999
19jul1999
26jul1999
02aug1999
09aug1999
16aug1999
23aug1999
30aug1999
06sep1999
13sep1999
20sep1999
27sep1999
04oct1999
11oct1999
18oct1999
25oct1999
01nov1999
08nov1999
15nov1999
22nov1999
29nov1999
06dec1999
13dec1999
20dec1999
27dec1999
03jan2000
10jan2000
17jan2000
24jan2000
31jan2000
07feb2000
14feb2000
21feb2000
28feb2000
06mar2000
13mar2000
20mar2000
27mar2000
03apr2000
10apr2000
17apr2000
24apr2000
01may2000
08may2000
15may2000
22may2000
29may2000
05jun2000
12jun2000
19jun2000
26jun2000
03jul2000
10jul2000
17jul2000
24jul2000
31jul2000
07aug2000
14aug2000
21aug2000
28aug2000
04sep2000
11sep2000
18sep2000
25sep2000
02oct2000
09oct2000
16oct2000
23oct2000
30oct2000
06nov2000
13nov2000
20nov2000
27nov2000
